# Принципы обработки больших данных на примере https://mlbootcamp.ru/round/13/sandbox/
Притыковская Наташа, Mail.ru




__Большие данные – это то, что вы не можете втянуть в память на одной машине__

## Проверяем размеры

In [1]:
!du -h

 36K	./.ipynb_checkpoints
152M	./trainGraph
157M	.


__152 мегабайта - piece of cake!__

### Применительно к нашей задаче

* Первый прогноз можно построить по счетчику общих друзей
* Выданный граф не симетричный (по сути двудольный), чтобы удобно считать общих друзей нужно его развернуть
* Имея в памяти основной и развернутый графы задача решается просто

## Попытка №1

Рекомендуем в качестве друзей тех, у кого много общих друзей с нами. Для этого втягиваем в память словарь вида [я->мои друзья], и его же в "развернутом виде" [я->те у кого я в друзьях]

### Готовим окружение

In [2]:
import csv, gzip, os, psutil, random, numpy, math

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
resultsPath = os.path.join(dataPath, "results")

# Утилитка для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print "Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024)

## Попытка №1 - читаем данные

In [3]:
%%time
mineFriends = defaultdict(list)
friendsOfMine = defaultdict(list)

# Итерируемся по файлам в папке
for file in [f for f in os.listdir(graphPath) if f.startswith("part")]:
    csvinput = gzip.open(os.path.join(graphPath, file))
    csv_reader = csv.reader(csvinput, delimiter='\t')
    # А теперь по строкам в файле
    for line in csv_reader:
        user = int(line[0]) 
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            friend = int(parts[0])
            mineFriends[user] += [friend]
            friendsOfMine[friend] += [user]

CPU times: user 2min 45s, sys: 7.17 s, total: 2min 52s
Wall time: 2min 57s


## Попытка №1 - тревожные сигналы

2 минуты на загрузку 152 мегабайт явно долговато, а что по памяти?

In [4]:
memory_usage()

Memory usage 2462 mb.


Все бы ничего, но два словарика в памяти __уже__ заняли больше трех гигабайт и дальше ворочать их будет неудобно. Но поробуем...

## Попытка №1 - пробуем расчет

In [5]:
%%time
friendsOfFriends = defaultdict(lambda: defaultdict(int))

for (user,friends) in mineFriends.iteritems():
    for friend in friends:
        for theirFriend in friendsOfMine[friend]:
            friendsOfFriends[user][theirFriend] += 1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/pritykovskaya/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/pritykovskaya/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Users/pritykovskaya/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/pritykovskaya/anaconda/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/pritykovskaya/anaconda/lib/python2.7/inspect.py", line 1008, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/pritykovskaya/anaconda/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodul

IndexError: string index out of range

## Попытка №1 - работа над ошибками

In [7]:
del friendsOfFriends
del mineFriends
del friendsOfMine

NameError: name 'friendsOfFriends' is not defined

In [8]:
memory_usage()

Memory usage 1576 mb.


### Применительно к нашей задаче

* Оценим объем данных
* Не забудем о накладных расходах!

## Разбор полетов - исследуем данные

Если с ходу не получилось, значит пришло время задуматся о том, как использовать память эффективнее. Для начала давайте попробуем прикинуть базовые показатели графа с которым работаем.

In [9]:
numUsersFrom = 0
numLinks = 0
maxUserIdFrom = 0
maxUserIdTo = 0
allIds = set()

for file in [f for f in os.listdir(graphPath) if f.startswith("part")]:
    csvinput = gzip.open(os.path.join(graphPath, file))
    csv_reader = csv.reader(csvinput, delimiter='\t')
    for line in csv_reader:
        user = int(line[0])
        numUsersFrom += 1
        maxUserIdFrom = max(user,maxUserIdFrom)
        allIds.add(user)
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            friend = int(parts[0])
            allIds.add(friend)
            numLinks += 1
            maxUserIdTo = max(maxUserIdTo,friend)
            
print("Num users from {0}, max {1}, num total {2}, max {3}, num links {4}".format(
        numUsersFrom,maxUserIdFrom,len(allIds),maxUserIdTo,numLinks))

Num users from 107474, max 16619036, num total 12417564, max 16619131, num links 36192484


### Оценим объем нужной памяти

На одну связь надо 2 идишки, которые 4-х байтовый инт

In [10]:
numLinks * (4 + 4) / 1024 / 1024

276

__Почему вместо 300Мб мы получили сразу 3Гб? Почему??!!__

## Разбор полетов - лезем в код

```c
#define PyDict_MINSIZE 8
typedef struct _dictobject PyDictObject;
struct _dictobject {
  ...
  PyDictEntry *ma_table;
  ...
};

typedef struct {
  Py_ssize_t me_hash;
  PyObject *me_key;
  PyObject *me_value;
} PyDictEntry;
```

## Хэш-таблица с открытой адресацией

<img src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg" width="800" />

## Разбор полетов - считаем накладные расходы

* На запись
  * +8 байт на указатель на запись
  * +4 байта на хэш
  * +8*2 байтов на указатели на ключ/значение
* +30% свободных ячеек
* х2 свободных ячеек при копировании

In [11]:
(numLinks * ((4 + 4) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

1960.4998504638672

Становится яснее!

## А ведь есть еще вопрос локальности данных!

<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/71982/images/2310246/figure04-caching-pyramid.png" width="600"/>
</center>

| Source   | Latency       |
| ---------|:-------------:|
| L1 Cache |  1-2 ns       |
| L2 Cache |  3-5 ns       |
| L3 Cache |  10-40 ns     |
| DRAM     |  60-100 ns    |



## Чем дальше в лес, тем толще партизаны - боксинг
```c
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

typedef struct {
    PyObject_HEAD
    long ob_ival;
} PyIntObject;
```

Кладя инт в "коробку" получаем 20 байт вместо 4-х, итого:

In [ ]:
(numLinks * ((20 + 20) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

## Разбор полетов - все проблемы вскрыли?

__Проблема заполнености__ - общие друзья есть у гораздо большего числа пар пользователей.

In [ ]:
numUsersFrom * numUsersFrom * 4 / 1024 / 1024

### Применительно к нашей задаче

* Считаем общих друзей только там, где это нужно (входят в тестовое множество)

## Попытка №2

* Выбираем структуру данных
* Втягиваем правильно
* Считаем общих друзей для целевых юзеров
* Сабмитим
* Профит!

## Выбираем структуры данных

### Словари

* Легко пользоватся.
* Но _ОЧЕНЬ_ много накладных расходов: указатель на запись, указатель на ключ/значение, объектная обертка для ключа и значения.
* Итого: на пару инт-инт _накладных_ расходов 48 байт (facepalm).
* Данные размазаны по памяти и фрагментированны - кеш процессора резко терят эффективность.

### Списки

* Если ключ инт - можно использовать вместо словаря.
* НО! Список в питоне это массив указателей, а значит накладные расходы на указатель и объектную обертку ключа. остаются - 20 байт накладных расходов на запись :(.
* Проблема с потерей эффективности кеша сохраняется.

### Массивы numpy

* На пару инт-инт нужно всего 4 байта.
* Кеш процессора работает.
* Но нужно чтобы ключи шли последовательно (к счастью, в наших даных это так почти так: максимальный ид 16619131 при общем числе 12417564).
* Для того чтобы заполнить массив _очень_ желательно знать его размер изначально.
* Если делать двумерный массив 100к на 16м - будет очень, очень, очень больно...

### Спарс матрицы scipy

* Предназначены для хранения двухмерных матриц с очень большим количеством нулей.
* Эффективные варианты хранят в виде нескольких массивов numpy.
* COO - три массива одинаковой длинны моделируют списов троек [(i,j,v)]. Просто строить, легко интегрировать с пандой.
* CSR - один маленьки массив с индексом начала для каждого ряда (монотонно растет), и два массива равно длинны с ид колонки и значением: [{i : [(j,v)]}]. Более компактна в памяти, быстрые опреации.
* CSC - тоже самое что CSR, но индексированны колонки.

#### CSR matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csr_matrix.png" width="800"/>

#### CSC matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csc_matrix2.png" width="800"/>

## Попытка № 2 - наш выбор

* Инициализруем граф в виде COO матрицы, затем переводим в CSR.
* Развернутый граф получаем с помощью транспонирования.
* Общих друзей считаем умножая матрицы.

#### Умножение матриц? WTF?

<img src="https://www.mathsisfun.com/algebra/images/matrix-multiply-a.gif" width="800"/>

## Попытка №2 - готовим окружение

In [12]:
# Почистим шлак и оставим константы чтоб потом не пресчитывать
del allIds
maxUserId = 16619131
numLinks = 36192484

In [13]:
# Эффективные массивы простых типов
import numpy
# Работа с матрицами (подсчет общих друзей реализован как умножение матрицы графа самое на себя)
import scipy
from scipy.sparse import coo_matrix, csr_matrix

## Попытка №2 - читаем матрицу

In [14]:
formUser = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
toUser = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
data = numpy.ones( (numLinks), dtype=numpy.int16 ) 

current = 0

# Итерируемся по файлам в папке... Да, опять. God bless Cmd+C/Cmd+V
for file in [f for f in os.listdir(graphPath) if f.startswith("part")]:
    csvinput = gzip.open(os.path.join(graphPath, file))
    csv_reader = csv.reader(csvinput, delimiter='\t')
    for line in csv_reader:
        user = int(line[0]) 
        maxUserId = max(user,maxUserId)
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            # Записываем связь в массивы и двигаем указатель
            formUser[current] = user
            friend = int(parts[0])
            toUser[current] = friend
            maxUserId = max(friend,maxUserId)
            current += 1
            
    csvinput.close()

## Попытка №2 - читаем матрицу

Собственно из массивов создаем COO матрицу

In [15]:
fullMatrix = coo_matrix(
    (data, (formUser, toUser)),
    shape=(maxUserId + 1, maxUserId + 1)).tocsr()

В матрице три массива, соответственно легко померять сколько памяти она занимает

In [16]:
print fullMatrix.data.nbytes + fullMatrix.indptr.nbytes + fullMatrix.indices.nbytes

283631436


Теперь аккуратно вычистим ненужное

In [17]:
del formUser
del toUser
del data  

## Попытка №2 - разворачиваем матрицу

In [19]:
reversedMatrix = scipy.transpose(fullMatrix).tocsr()

Есть развернутая матрица, проверим сколько занимает

In [ ]:
print reversedMatrix.data.nbytes + reversedMatrix.indptr.nbytes + reversedMatrix.indices.nbytes

Итого - две матрицу в сумме 540Мб в памяти :). Пора считать общих друзей?


* Сохраним полученные результаты (граф и развернутый граф) в бинарном формате, чтобы не тратить время на чтение текстов.
* Далее будем сохранять все сложновычисляемые результаты.

## Определим функции для сохранения и загрузки матриц

In [20]:
def save_csr(matrix, path):
    numpy.savez(os.path.join(resultsPath, path),
            data=matrix.data,
            indices=matrix.indices,
            indptr=matrix.indptr,
            shape=matrix.shape)

def load_csr(path):
    loaded = numpy.load(os.path.join(resultsPath, path + ".npz"))
    return csr_matrix(
        (loaded['data'], loaded['indices'], loaded['indptr']), 
        shape=loaded['shape'])

## Сохраняем и тестируем загрузку

In [22]:
%%time
save_csr(fullMatrix, "fullMatrix")
save_csr(reversedMatrix, "reversedMatrix")

CPU times: user 2.09 s, sys: 2.97 s, total: 5.05 s
Wall time: 6.14 s


In [23]:
%%time
fullMatrix = load_csr("fullMatrix")
reversedMatrix = load_csr("reversedMatrix")

CPU times: user 1.51 s, sys: 483 ms, total: 1.99 s
Wall time: 2.13 s


## Попытка №2 - готовимся считать только для нужных юзеров

Читаем ид тех, для кого нужен прогноз

In [ ]:
validationUsers = set()
csv_reader = csv.reader(open(os.path.join(dataPath, "prediction.csv")))
for line in csv_reader: validationUsers.add(int(line[0]))

Занулим чужих пользователей

In [ ]:
for i in range(maxUserId + 1):
    if i not in validationUsers:
        ptr = fullMatrix.indptr[i]
        ptr_next = fullMatrix.indptr[i+1]
        if ptr != ptr_next:
            fullMatrix.data[ptr:ptr_next].fill(0)
            

Вычистим нули

In [ ]:
fullMatrix.eliminate_zeros()

## Не забываем сохранить

In [ ]:
save_csr(fullMatrix, "validationUsersGraph")

In [ ]:
validationUsersGraph = load_csr("validationUsersGraph")

In [ ]:
print validationUsersGraph.data.nbytes + validationUsersGraph.indptr.nbytes + validationUsersGraph.indices.nbytes

## Попытка №2 - считаем общих друзей

In [ ]:
%%time
commonFriends = validationUsersGraph.dot(reversedMatrix)

Интересно, сколько получилось в памяти?

In [ ]:
print commonFriends.data.nbytes + commonFriends.indptr.nbytes + commonFriends.indices.nbytes

Не забыть сохранить!

In [ ]:
save_csr(commonFriends, "validation_commonFriends")

## Попытка №2 - генерируем прогноз

In [ ]:
%%time
output = open( os.path.join(resultsPath, "prediction.csv"), "w")

for i in validationUsers:
    ptr = commonFriends.indptr[i]
    ptr_next = commonFriends.indptr[i+1] 
    if ptr != ptr_next:
        counts = commonFriends.data[ptr:ptr_next]
        order = numpy.argsort(-counts)
        
        # Не забываем что из прогноза надо убрать себя и своих известных друзей
        mineFriends = set(fullMatrix.indices[fullMatrix.indptr[i]:fullMatrix.indptr[i+1]])
        mineFriends.add(i)
        
        ids = commonFriends.indices[ptr:ptr_next]
        prediction = filter(lambda x: x not in mineFriends, ids[order])
        output.write(str(i) + ",\"" + " ".join(map(str, prediction)) + "\"\n")      
        
# Не забываем закрыть файл
output.close()        

## Хм, как долго-то... И сколько получилось в итоге?

In [ ]:
print os.path.getsize(os.path.join(resultsPath, "prediction.csv")) / 1024 / 1024

Мда, 340Мб, не влезет ни в какие лимиты. Снова применим _4-й принцип_ - надо рубить хвосты!

## Попытка № 3 - уменьшаем объем результата

In [ ]:
%%time
output = open( os.path.join(resultsPath, "prediction_short.csv"), "w")

for i in validationUsers:
    ptr = commonFriends.indptr[i]
    ptr_next = commonFriends.indptr[i+1] 
    if ptr != ptr_next:
        counts = commonFriends.data[ptr:ptr_next]
        order = numpy.argsort(-counts)
        mineFriends = set(fullMatrix.indices[fullMatrix.indptr[i]:fullMatrix.indptr[i+1]])
        mineFriends.add(i)   
        ids = commonFriends.indices[ptr:ptr_next]
        # И собственно фильтр. 42
        prediction = filter(lambda x: x not in mineFriends, ids[order])[:42]
        output.write(str(i) + ",\"" + " ".join(map(str, prediction)) + "\"\n")
    else:
        output.write(str(i) + ",\"1447358 665059 4554374\"\n")

# Не забываем закрыть файл
output.close() 

Гораздо быстрее! И сколько же там?

In [ ]:
print os.path.getsize(os.path.join(resultsPath,"prediction_short.gz"))